# Preparing for Machine Learning

In [13]:
import pyodbc

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=GOOSEBOX;"
    "DATABASE=NFL;"
    "Trusted_Connection=yes;"
    "Encrypt=yes;"
    "TrustServerCertificate=yes;"
)
print("Connection successful!")
conn.close()


Connection successful!


In [15]:
from sqlalchemy import create_engine
import urllib

# Define the connection parameters
params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=GOOSEBOX;"
    "DATABASE=NFL;"
    "Trusted_Connection=yes;"
    "Encrypt=yes;"
    "TrustServerCertificate=yes;"
)

# Create the SQLAlchemy engine
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Test the connection
try:
    with engine.connect() as connection:
        print("SQLAlchemy connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")


SQLAlchemy connection successful!


Note: Polars cannot read directly from a SQL Database, but since the data has been shrunk at this point, we can continue the 
analysis in Pandas, rather than converting from pandas to polars and back to pandas again in creating new ML databases. 

In [19]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

# Define your SQL query
query = "SELECT * FROM dbo.Injury_Summary"

# Use Pandas to read the SQL query into a DataFrame
with engine.connect() as connection:
    pandas_df = pd.read_sql_query(query, connection)

In [20]:
len(pandas_df)

266594

In [23]:
pandas_df.columns

Index(['PlayKey', 'Position', 'StadiumType', 'FieldType', 'Temperature',
       'Weather', 'PlayType', 'BodyPart', 'DM_M1', 'DM_M7', 'DM_M28', 'DM_M42',
       'IsInjured', 'IsSevere', 'Position_right', 'Distance', 'Displacement',
       'Path_Diff', 'Max_Angle_Diff', 'Mean_Angle_Diff', 'Max_Speed',
       'Mean_Speed', 'Max_Impulse', 'Mean_Impulse', 'Max_Torque',
       'Mean_Torque', 'Max_Int_Torque', 'Mean_Int_Torque'],
      dtype='object')